In [5]:
import pandas as pd
from selenium import webdriver

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import time
import os

from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [6]:
navigator = webdriver.Chrome(ChromeDriverManager().install())
navigator.get(os.getenv('URL'))

/tmp/ipykernel_16524/1960473131.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navigator = webdriver.Chrome(ChromeDriverManager().install())


In [9]:
print(navigator.find_element(By.XPATH, os.getenv('TITULO-LEILAO')).text)

LEILÃO DE VEÍCULOS NORDESTE - 08/03/23


In [ ]:
# !pip install webdriver-manager